In [1]:
import os, subprocess , IPython
import matplotlib.pyplot as plt
import numpy as np

import essentia.standard as es
#import essentia.streaming

import pylab  #plot, show, figure, imshow
import cv2
import moviepy.editor as mp

import utils.util as util

2023-04-10 23:44:38.306393: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
''' HELPERS '''
#print(dir(essentia.standard))
#print(help(es.AudioLoader))

In [ ]:
#mp4_paths,*_ = util.load_xdv_test(aac_path=util.SERVER_TEST_AUD_MONO_PATH)
#out2 = util.print_acodec_from_mp4(mp4_paths,printt=True)

In [ ]:
#mp4_paths,mp4_labels,aac_paths,aac_labels = util.load_xdv_test(aac_path=util.SERVER_TEST_AUD_MONO_PATH)
#aacpath = aac_paths[44]
#mp4path = mp4_paths[44]

#IPython.display.Audio(aacpath)

In [ ]:
''' writes 10 sec of audio from mp4 cv read synced'''
def get_audio_from_mp4(path,start_t=0.0,end_t=10.0):

    video = cv2.VideoCapture(path)
    fps = video.get(cv2.CAP_PROP_FPS)

    # Extract the audio from the video
    audio = mp.AudioFileClip(path)
    
    while True:
        
        ret, frame = video.read()
        if not ret:break
        video_atual_frame = int(video.get(cv2.CAP_PROP_POS_FRAMES))
        
        start=video.get(cv2.CAP_PROP_POS_MSEC)/1000
        end=(video.get(cv2.CAP_PROP_POS_MSEC)+1000/fps)/1000
        print("1 frame sec start end",start,end)
        
        if video_atual_frame == 240:

            start=start_t
            end=end_t
            print(start,end)
            
            audio_frame = audio.subclip(t_start=start,t_end=end)
            audio_array = np.array(audio_frame.to_soundarray())
            
            nsamples = np.shape(audio_array)[0]
            secs = end-start
            sample_rate = nsamples / secs
            print(audio_array.dtype,np.shape(audio_array) , sample_rate)
            
            # 1 transform stereo to mono
            audio_array_mono = np.mean(audio_array, axis=1)
            es.MonoWriter(filename='audio_mono.wav',sampleRate = int(sample_rate))(audio_array_mono)
            
            # 2 save as stereo
            es.AudioWriter(filename='audio_stereo.wav',sampleRate = int(sample_rate))(audio_array)
            
            break

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()
    
    
#get_audio_from_mp4(mp4path)

In [4]:
import os


# Define the file paths from the previous list and the new list
previous_list = [
    "/raid/DATASETS/anomaly/XD_Violence/training_copy/v=IhU6jml6NY4__#1_label_G-0-0.mp4",
    "/raid/DATASETS/anomaly/XD_Violence/training_copy/v=abc123#1_label_G-0-0.mp4",
    "/raid/DATASETS/anomaly/XD_Violence/training_copy/v=def456#1_label_G-0-0.mp4"
]
new_list = [
    "/raid/DATASETS/anomaly/XD_Violence/training_copy/v=xyz789#1_label_G-0-0.mp4",
    "/raid/DATASETS/anomaly/XD_Violence/training_copy/v=abc123#1_label_G-0-0.mp4"
]

# Extract the video identifier from each file path in the new list
new_video_identifiers = set()
for new_path in new_list:
    video_identifier = os.path.basename(new_path).split("_")[0].split("=")[1]
    new_video_identifiers.add(video_identifier)

# Loop through the previous list and exclude files with matching video identifiers
exclude_list = []
for prev_path in previous_list:
    video_identifier = os.path.basename(prev_path).split("_")[0].split("=")[1]
    if video_identifier in new_video_identifiers:
        exclude_list.append(prev_path)

# Create a filtered list of file paths from the new list
filtered_list = [new_path for new_path in new_list
                 if os.path.basename(new_path) not in exclude_list]

# Print the filtered file paths
print("Filtered file paths:")
for filtered_path in filtered_list:
    print(filtered_path)

Filtered file paths:
/raid/DATASETS/anomaly/XD_Violence/training_copy/v=xyz789#1_label_G-0-0.mp4
/raid/DATASETS/anomaly/XD_Violence/training_copy/v=abc123#1_label_G-0-0.mp4
